# Accidents cargo

## Import des dépendances et du dataset

In [ ]:
import pandas as pd
import geopandas as gpd
import datetime
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Chemin vers le .shp
shp_path = "../data/extracted/Shipping_Accidents/Shipping_Accidents.shp"

# Lecture du shapefile avec geopandas
df = gpd.read_file(shp_path)

## Préparation des données

### Nettoyage des données

1. Les données allant des années 1989 à 2023, nous allons les filtrer et ne garder qu'une intervalle de 20 ans, soit de 2003 à 2023. Cela nous permettra de ne pas avoir des données trop anciennes qui pourraient fausser notre analyse.

2. Le nombre de types d'accidents étant important voir redondant, nous allons les regrouper en 5 catégories :
   - Technical or Equipment Failure
   - Navigation or Maneuvering Incident
   - Fire or Explosion
   - Life-saving Equipment Incident
   - Other

In [ ]:
## Définition de l'intervalle de temps
start_year = 2003
end_year = 2023

df = df[(df['Year'] >= start_year) & (df['Year'] <= end_year)]


## Suppression des coordonnées incohérentes
# Suppression des lignes où les coordonnées sont nulles ou égales à 0
df = df[~((df['Longitude'].isnull()) | (df['Latitude'].isnull()) | (df['Longitude'] == 0) | (df['Latitude'] == 0))]


## Regroupement des types d'accidents
# Défaillance technique ou équipement
df['Acc_Type'] = df['Acc_Type'].replace('Damage to ship or equipment', 'Défaillance technique ou équipement')
df['Acc_Type'] = df['Acc_Type'].replace('Damages to ships or equipment', 'Défaillance technique ou équipement')
df['Acc_Type'] = df['Acc_Type'].replace('Dammage to ships or equipment', 'Défaillance technique ou équipement')
df['Acc_Type'] = df['Acc_Type'].replace('Door fault . fault in doorways', 'Défaillance technique ou équipement')
df['Acc_Type'] = df['Acc_Type'].replace('hull failure', 'Défaillance technique ou équipement')
df['Acc_Type'] = df['Acc_Type'].replace('Hull failure/failure of watertight doors/ports etc.', 'Défaillance technique ou équipement')
df['Acc_Type'] = df['Acc_Type'].replace('machinery damage', 'Défaillance technique ou équipement')
df['Acc_Type'] = df['Acc_Type'].replace('Machinery damage', 'Défaillance technique ou équipement')
df['Acc_Type'] = df['Acc_Type'].replace('Machinery dammage', 'Défaillance technique ou équipement')
df['Acc_Type'] = df['Acc_Type'].replace('macihnery damage', 'Défaillance technique ou équipement')
df['Acc_Type'] = df['Acc_Type'].replace('Technical failure', 'Défaillance technique ou équipement')

# Navigation or Maneuvering Incident
df['Acc_Type'] = df['Acc_Type'].replace('Capsizing.listing', 'Erreur de navigation ou de manoeuvre')
df['Acc_Type'] = df['Acc_Type'].replace('Capsizing/listing', 'Erreur de navigation ou de manoeuvre')
df['Acc_Type'] = df['Acc_Type'].replace('collision', 'Erreur de navigation ou de manoeuvre')
df['Acc_Type'] = df['Acc_Type'].replace('Collision', 'Erreur de navigation ou de manoeuvre')
df['Acc_Type'] = df['Acc_Type'].replace('contact', 'Erreur de navigation ou de manoeuvre')
df['Acc_Type'] = df['Acc_Type'].replace('Contact', 'Erreur de navigation ou de manoeuvre')
df['Acc_Type'] = df['Acc_Type'].replace('Flooding/Foundering', 'Erreur de navigation ou de manoeuvre')
df['Acc_Type'] = df['Acc_Type'].replace('grounding', 'Erreur de navigation ou de manoeuvre')
df['Acc_Type'] = df['Acc_Type'].replace('Grounding', 'Erreur de navigation ou de manoeuvre')
df['Acc_Type'] = df['Acc_Type'].replace('Grounding/stranding', 'Erreur de navigation ou de manoeuvre')
df['Acc_Type'] = df['Acc_Type'].replace('Loss of control', 'Erreur de navigation ou de manoeuvre')
df['Acc_Type'] = df['Acc_Type'].replace('stranding.grounding', 'Erreur de navigation ou de manoeuvre')
df['Acc_Type'] = df['Acc_Type'].replace('Stranding.grounding', 'Erreur de navigation ou de manoeuvre')
df['Acc_Type'] = df['Acc_Type'].replace('stranding/grounding', 'Erreur de navigation ou de manoeuvre')
df['Acc_Type'] = df['Acc_Type'].replace('Stranding/grounding', 'Erreur de navigation ou de manoeuvre')
df['Acc_Type'] = df['Acc_Type'].replace('Tilt / crash', 'Erreur de navigation ou de manoeuvre')

# Fire or Explosion
df['Acc_Type'] = df['Acc_Type'].replace('Fire', 'Incendie ou explosion')
df['Acc_Type'] = df['Acc_Type'].replace('Fire . explosion', 'Incendie ou explosion')
df['Acc_Type'] = df['Acc_Type'].replace('Fire/Explosion', 'Incendie ou explosion')
df['Acc_Type'] = df['Acc_Type'].replace('Fire or explosion', 'Incendie ou explosion')


# Life-saving Equipment Incident
df['Acc_Type'] = df['Acc_Type'].replace('Accidents with life-saving appliances', 'Équipement de sauvetage')
df['Acc_Type'] = df['Acc_Type'].replace('Related to the use of rescue equipment', 'Équipement de sauvetage')

# Other
df['Acc_Type'] = df['Acc_Type'].replace('n.i.', 'Autre')
df['Acc_Type'] = df['Acc_Type'].replace('other', 'Autre')
df['Acc_Type'] = df['Acc_Type'].replace('Other', 'Autre')
df['Acc_Type'] = df['Acc_Type'].replace("other (unsealing the vessel's hull)", 'Autre')
df['Acc_Type'] = df['Acc_Type'].replace('Other reason', 'Autre')
df['Acc_Type'] = df['Acc_Type'].replace('Other type', 'Autre')
df['Acc_Type'] = df['Acc_Type'].replace('Physical damage', 'Autre')
df['Acc_Type'] = df['Acc_Type'].replace('Sunk', 'Autre')
df['Acc_Type'] = df['Acc_Type'].replace('v.serious accident', 'Autre')
df['Acc_Type'] = df['Acc_Type'].fillna('Autre')

### Enrichissement des données

La colonne `Location` n'étant pas toujours renseignée, nous allons la compléter en utilisant deux autres datassets :
- World Port Index – Port Data
- Natural Earth
Ces deux datasets contiennent des informations sur les ports et les côtes du monde entier, ce qui nous permettra par croisement de données de compléter les informations manquantes dans notre dataset.

In [ ]:
# Conversion du DataFrame en GeoDataFrame avec CRS standard
df.to_crs("EPSG:4326", inplace=True)
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

# Chargement des shapefiles
lands = gpd.read_file("../data/extracted/Shipping_Accidents/lands.shp")
ports = gpd.read_file("../data/extracted/Shipping_Accidents/ports.shp")

# Forcer le CRS si non défini
for layer in [lands, ports]:
    if layer.crs is None:
        layer.set_crs("EPSG:4326", inplace=True)

# Reprojection en projection métrique (pour calculs de distances)
gdf_proj = gdf.to_crs("EPSG:3857")
ports_proj = ports.to_crs("EPSG:3857")
land_proj = lands.to_crs("EPSG:3857")

# Création des buffers
buffer_distances = {
    "port": 3000,        # 3 km
    "approach": 10000,   # 10 km
    "coast": 20000       # 20 km
}

ports_buffer = gpd.GeoDataFrame(geometry=ports_proj.buffer(buffer_distances["port"]), crs=ports_proj.crs)
ports_approach_buffer = gpd.GeoDataFrame(geometry=ports_proj.buffer(buffer_distances["approach"]), crs=ports_proj.crs)
coast_buffer = gpd.GeoDataFrame(geometry=land_proj.buffer(buffer_distances["coast"]), crs=land_proj.crs)


## Attribution des zones géographiques par jointures spatiales

# a) Zone portuaire
join_port = gdf_proj.sjoin(ports_buffer, how="left", predicate="intersects")
is_port = pd.Series(False, index=gdf_proj.index)  # Série de False par défaut
is_port.loc[join_port.index] = join_port["index_right"].notnull()
gdf_proj["is_port"] = is_port

# b) Zone d’approche portuaire (hors port)
join_approach = gdf_proj[~gdf_proj["is_port"]].sjoin(ports_approach_buffer, how="left", predicate="intersects")
is_approach = pd.Series(False, index=gdf_proj.index)
is_approach.loc[join_approach.index] = join_approach["index_right"].notnull()
gdf_proj["is_port_approach"] = is_approach

# c) Zone côtière (hors port et approche)
mask = (~gdf_proj["is_port"]) & (~gdf_proj["is_port_approach"].fillna(False))
join_coast = gdf_proj[mask].sjoin(coast_buffer, how="left", predicate="intersects")
is_coastal = pd.Series(False, index=gdf_proj.index)
is_coastal.loc[join_coast.index] = join_coast["index_right"].notnull()
gdf_proj["is_coastal"] = is_coastal

# d) Classification finale de l'emplacement
def classify_location(row):
    if row["is_port"]:
        return "Port"
    elif row["is_port_approach"]:
        return "En approche du port"
    elif row["is_coastal"]:
        return "Mer"
    else:
        return "Haute mer"

gdf_proj["Location"] = gdf_proj.apply(classify_location, axis=1)


## Reprojection finale en WGS84 + mise à jour du DataFrame d’origine

df["Location"] = gdf_proj.to_crs("EPSG:4326")["Location"].values

In [ ]:
# Seuil strict de suppression
threshold = 0.85

# Identification et suppression
cols_to_drop = df.columns[df.isnull().mean() > threshold]
df.drop(columns=cols_to_drop, inplace=True)

print(f"Colonnes supprimées (> {int(threshold*100)}% de valeurs manquantes) :")
print(list(cols_to_drop))

In [ ]:
# Nettoyage + remplissage intelligent des colonnes critiques

df["Damage"] = (
    df["Damage"]
    .str.strip().str.title()
    .replace({"Not Known": None, "Unknown": None, "": None})
    .fillna("Non renseigné")
)

df["Pollu_t"] = (
    df["Pollu_t"]
    .str.strip().str.lower()
    .replace({"not known": None, "unknown": None, "": None})
    .fillna("non précisé")
)

df["Cause_Sh1"] = (
    df["Cause_Sh1"]
    .str.strip().str.capitalize()
    .replace({"": None})
    .fillna("Non précisé")
)

df["Pilot_Sh1"] = (
    df["Pilot_Sh1"]
    .str.strip().str.lower()
    .replace({"not known": None, "unknown": None, "": None})
    .fillna("inconnu")
)

df["Assistance"] = (
    df["Assistance"]
    .str.strip().str.capitalize()
    .fillna("Non renseigné")
)

df["Ship1_Name"] = (
    df["Ship1_Name"]
    .str.strip().str.upper()
    .fillna("NON RENSEIGNE")
)

df["Colli_Type"] = df["Colli_Type"].str.strip().str.title().fillna("Non précisé")
df["Cargo_Type"] = df["Cargo_Type"].str.strip().str.title().fillna("Non précisé")
df["IceCondit"] = df["IceCondit"].str.strip().str.title().fillna("Non précisé")
df["Sh1_Type"] = df["Sh1_Type"].str.strip().str.title().fillna("Non précisé")


In [ ]:
# Suppression stricte des colonnes avec trop de valeurs manquantes restantes
cols_to_remove_final = [
    "CauseDetai", "CrewIceTra", "HumanEleme", "Sh2Size_gt",
    "Sh2_Categ", "Acc_Detail", "Sh1_Hull", "Date"
]
df.drop(columns=cols_to_remove_final, inplace=True)

print("Colonnes définitivement supprimées car trop incomplètes :")
print(cols_to_remove_final)

In [ ]:
# Ajout des colonnes
df["Decade"] = (df["Year"] // 10) * 10

df["Damage_Severe"] = df["Damage"].isin(["Severe Damage", "Total Loss"]).astype(int)

location_map = {
    "Port": "P",
    "En approche du port": "A",
    "Mer": "S",
    "Haute mer": "O"
}
df["Location_Code"] = df["Location"].map(location_map)


In [ ]:
df["Pollution"] = df["Pollution"].str.strip().str.lower()
df["Pollution"] = df["Pollution"].replace({
    "yes": "Oui",
    "no": "Non",
    "n.i.": "Non précisé"
})

# Sauvegarde brute si besoin
df["Sh1Draug_raw"] = df["Sh1Draug_m"]

# Nettoyage contrôlé sans perdre les autres données
df["Sh1Draug_m"] = df["Sh1Draug_m"].replace("n.i.", "Non précisé")

# Conversion avec fallback texte
df["Sh1Draug_m"] = df["Sh1Draug_m"].apply(lambda x: float(x) if str(x).replace('.', '', 1).isdigit() else "Non précisé")


In [ ]:
# Nettoyage texte
df["Damage_clean"] = df["Damage"].str.strip().str.lower()

# Regroupement par classes
def classify_damage(text):
    if pd.isnull(text) or text in ["non renseigné", "no damage", "0", "n.i."]:
        return "Aucun"
    elif any(kw in text for kw in ["minor", "light", "scratches", "superficial"]):
        return "Mineur"
    elif any(kw in text for kw in ["damage", "fracture", "hull", "propeller", "fire"]):
        return "Modéré"
    elif any(kw in text for kw in ["severe", "flood", "sinking", "explosion", "total", "major"]):
        return "Sévère"
    else:
        return "Inconnu"

df["Damage_Class"] = df["Damage_clean"].apply(classify_damage)

In [ ]:
# Nettoyage des valeurs textuelles
df["Pollution"] = df["Pollution"].str.strip().str.lower()
df["Pollution"] = df["Pollution"].replace({
    "yes": "Oui",
    "oui": "Oui",
    "no": "Non",
    "non": "Non",
    "n.i.": "Non précisé",
    "no information": "Non précisé",
    "unknown": "Non précisé"
})

# Création colonne binaire : Pollution présente ou non
df["Pollution_Binaire"] = df["Pollution"].apply(lambda x: 1 if x == "Oui" else 0 if x == "Non" else None)

In [ ]:
# Nettoyage des champs pollution volume
df["Pollu_t_clean"] = pd.to_numeric(
    df["Pollu_t"].astype(str).str.replace(",", ".").str.extract(r"([\d.]+)")[0],
    errors="coerce"
)
df["Pollu_m3_clean"] = pd.to_numeric(
    df["Pollu_m3"].astype(str).str.replace(",", ".").str.extract(r"([\d.]+)")[0],
    errors="coerce"
)

# Score simple : somme des 2 (à adapter selon besoin)
df["Pollution_Score"] = df["Pollu_t_clean"].fillna(0) + df["Pollu_m3_clean"].fillna(0)

# Ajout d'un indicateur de complétion du Pollution_Score
df["Pollution_Score_Complet"] = df[["Pollu_t_clean", "Pollu_m3_clean"]].notnull().all(axis=1).astype(int)

# Score pondéré en fonction de la complétion
df["Pollution_Score_Weighted"] = df["Pollution_Score"] * df["Pollution_Score_Complet"]

# Catégorie de fiabilité du score
df["Pollution_Qualité"] = df["Pollution_Score_Complet"].map({1: "Fiable", 0: "Partiel"})

In [ ]:
# Nettoyage du champ taille du navire
df["Sh1Size_gt_clean"] = pd.to_numeric(
    df["Sh1Size_gt"].astype(str).str.replace(",", ".").str.extract(r"([\d.]+)")[0],
    errors="coerce"
)

# Création du score brut (score direct égal à la taille)
df["Ship_Profile_Score"] = df["Sh1Size_gt_clean"].fillna(0)

# Catégorisation des tailles de navire : seuils arbitraires à ajuster si besoin
def categorize_ship_size(gt):
    if pd.isna(gt):
        return "Inconnu"
    elif gt < 3000:
        return "Petit"
    elif gt < 15000:
        return "Moyen"
    else:
        return "Grand"

df["Ship_Profile_Class"] = df["Sh1Size_gt_clean"].apply(categorize_ship_size)

In [ ]:
# Transformation de l'heure (Time) pour catégorisation
def classify_time_of_day(time_str):
    try:
        # Conversion vers objet time
        t = datetime.datetime.strptime(time_str, "%I:%M:%S %p").time()
        if t >= datetime.time(6, 0) and t < datetime.time(12, 0):
            return "Matin"
        elif t >= datetime.time(12, 0) and t < datetime.time(18, 0):
            return "Après-midi"
        elif t >= datetime.time(18, 0) and t < datetime.time(22, 0):
            return "Soir"
        else:
            return "Nuit"
    except:
        return None

# Application sur la colonne Time
df["Time_Period"] = df["Time"].apply(classify_time_of_day)

In [ ]:
# Catégorisation géographique simple
def categorize_latitude(lat):
    if lat < 56:
        return "Sud"
    elif 56 <= lat < 59:
        return "Centre"
    else:
        return "Nord"

def categorize_longitude(lon):
    if lon < 12:
        return "Ouest"
    elif 12 <= lon < 18:
        return "Centre"
    else:
        return "Est"

df["Geo_Latitude_Zone"] = df["Latitude"].apply(categorize_latitude)
df["Geo_Longitude_Zone"] = df["Longitude"].apply(categorize_longitude)
df["Geo_Zone"] = df["Geo_Latitude_Zone"] + "-" + df["Geo_Longitude_Zone"]

In [ ]:
# Uniformisation de Pollution_Qualité
df["Pollution_Qualité"] = df["Pollution_Qualité"].str.strip().str.capitalize()
df["Pollution_Qualité"] = df["Pollution_Qualité"].replace({
    "Partiel": "Partiel",
    "Partielle": "Partiel",
    "Fiable": "Fiable",
    "Reliable": "Fiable",
    "Non renseigné": None,
    "Non renseignée": None,
    "": None,
    None: None
})

# Uniformisation de Time_Period
df["Time_Period"] = df["Time_Period"].str.strip().str.capitalize()
df["Time_Period"] = df["Time_Period"].replace({
    "Matin": "Matin",
    "Morning": "Matin",
    "Après-midi": "Après-midi",
    "Apres-midi": "Après-midi",
    "Après Midi": "Après-midi",
    "Soir": "Soir",
    "Evening": "Soir",
    "Nuit": "Nuit",
    "Night": "Nuit",
    "": None,
    None: None
})


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Étape 1 : Variables utilisées
features = df[['Damage_Severe', 'Pollution_Score', 'Ship_Profile_Score']].copy()
features = features.fillna(0)

# Étape 2 : Normalisation robuste
scaler = MinMaxScaler()
features_scaled = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)

# Étape 3 : Pondérations métier
w_damage = 0.5
w_pollution = 0.35
w_ship = 0.15

# Étape 4 : Calcul du Risk Score + amplification pour lisibilité
df['Risk_Score'] = (
    w_damage * features_scaled['Damage_Severe'] +
    w_pollution * features_scaled['Pollution_Score'] +
    w_ship * features_scaled['Ship_Profile_Score']
)

# multiplier pour sortir du 0.0
df['Risk_Score'] *= 10

# Étape 5 : Classification par seuils métiers
def risk_classification(score):
    if score < 1:
        return 'Bas'
    elif score < 3:
        return 'Medium'
    elif score < 6:
        return 'Haut'
    else:
        return 'Critique'

df['Risk_Class'] = df['Risk_Score'].apply(risk_classification)

# Vérif console
print(df['Risk_Class'].value_counts(normalize=True))


In [ ]:
# Étape 2 : Classification équilibrée par quartiles
q = df['Risk_Score'].quantile([0, 0.25, 0.5, 0.75, 1]).values

def risk_classification_quantile(score):
    if score <= q[1]:
        return 'Bas'
    elif score <= q[2]:
        return 'Medium'
    elif score <= q[3]:
        return 'Haut'
    else:
        return 'Critique'

df['Risk_Class'] = df['Risk_Score'].apply(risk_classification_quantile)

# Vérif
print(df['Risk_Class'].value_counts(normalize=True).round(3))

In [ ]:
print(f"Low      : 0 ≤ score ≤ {q[1]:.3f}")
print(f"Medium   : {q[1]:.3f} < score ≤ {q[2]:.3f}")
print(f"High     : {q[2]:.3f} < score ≤ {q[3]:.3f}")
print(f"Critical : {q[3]:.3f} < score ≤ {q[4]:.3f}")

In [ ]:
# --- Audit complet et structuré du DataFrame ---

# 1. Taux de complétion par colonne (% de valeurs non manquantes)
print("Taux de complétion par colonne (%) :")
completion = (1 - df.isnull().mean()) * 100
print(completion.sort_values(ascending=True), "\n")

# 2. Nombre de valeurs uniques par colonne (hors NaN)
print("Colonnes avec des valeurs uniques :")
for col in df.columns:
    uniques = df[col].dropna().unique()
    print(f"{col}: {len(uniques)} valeurs uniques")
print()

# 3. Statistiques descriptives des colonnes numériques
print("Statistiques sur les colonnes numériques :")
print(df.select_dtypes(include=["number"]).describe().T, "\n")

# 4. Exemples de valeurs non numériques (max 5 exemples par colonne)
print("Exemples de valeurs non numériques :")
for col in df.select_dtypes(exclude=["number"]).columns:
    exemples = df[col].dropna().unique()[:5]
    print(f"{col} → Exemples : {exemples}")
print()

# 5. Répartition des classes pour la colonne 'Damage_Class'
print("Répartition des classes Damage_Class :")
print(df["Damage_Class"].value_counts(dropna=False), "\n")

# 6. Statistiques sur Pollution_Score
print("Statistiques Pollution_Score :")
print(df["Pollution_Score"].describe(), "\n")

# 7. Exemples de lignes où Pollution_Score > 0 (5 premiers)
print("Exemples Pollution_Score > 0 :")
print(df.loc[df["Pollution_Score"] > 0, ["Pollu_t", "Pollu_m3", "Pollution_Score"]].head(), "\n")

# 8. Valeurs uniques et répartition après uniformisation

print("✅ Taux de complétion et classes uniques après uniformisation :\n")

if "Pollution_Qualité" in df.columns:
    print("Pollution_Qualité :")
    print(df["Pollution_Qualité"].value_counts(dropna=False), "\n")

if "Time_Period" in df.columns:
    print("Time_Period :")
    print(df["Time_Period"].value_counts(dropna=False), "\n")

## Export du dataset préparé pour le dashboard

In [ ]:
colonnes_a_garder = [
    "Unique_ID",
    "Year", "Location", "Geo_Zone", "Geo_Latitude_Zone", "Geo_Longitude_Zone",
    "Damage_Class", "Damage_Severe", "Pollution_Score", "Pollution_Qualité", "Risk_Score", "Risk_Class",
    "Ship_Profile_Score", "Ship_Profile_Class", "Sh1Size_gt_clean",
    "Time_Period", "Decade",
    "Acc_Type", "Cargo_Type", "Colli_Type", "Assistance",
    "Latitude", "Longitude"
]

df_filtered = df[colonnes_a_garder]

df_filtered.to_csv("../data/cleaned/shipping_accidents_cleaned.csv", index=False)
print("Fichier enregistré : ../data/cleaned/shipping_accidents_cleaned.csv")